<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Take-notice!" data-toc-modified-id="Take-notice!-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Take notice!</a></span></li><li><span><a href="#Creating-a-function" data-toc-modified-id="Creating-a-function-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating a function</a></span><ul class="toc-item"><li><span><a href="#LA-Times-Neighborhoods" data-toc-modified-id="LA-Times-Neighborhoods-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LA Times Neighborhoods</a></span></li><li><span><a href="#Get-Arrest-Data-from-LA-Open-Data-Portal" data-toc-modified-id="Get-Arrest-Data-from-LA-Open-Data-Portal-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Get Arrest Data from LA Open Data Portal</a></span></li><li><span><a href="#Convert-data-to-a-geodataframe" data-toc-modified-id="Convert-data-to-a-geodataframe-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Convert data to a geodataframe</a></span></li><li><span><a href="#Create-a-two-layer-map" data-toc-modified-id="Create-a-two-layer-map-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Create a two layer map</a></span></li><li><span><a href="#Zoom-to-the-points,-not-the-neighborhoods" data-toc-modified-id="Zoom-to-the-points,-not-the-neighborhoods-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Zoom to the points, not the neighborhoods</a></span></li></ul></li><li><span><a href="#Function-time" data-toc-modified-id="Function-time-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Function time</a></span><ul class="toc-item"><li><span><a href="#Adding-more-arguments" data-toc-modified-id="Adding-more-arguments-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Adding more arguments</a></span></li></ul></li></ul></div>

<div class="alert alert-danger">

<h1>Take notice!</h1>
<ul>
    <li>Make sure you are working with a copy and not the original notebook file</li>
    <li>This class will be recorded</li>
</ul>
    
</div>

# Creating a function

In this lab, we use the Multiple Overlay lab from last week to learn how to convert a process—a chain of commands to produce a desired output—into a function. Functions are extremely useful, can save time, and cut your lines of code dramatically.

First, let's pick off from last week's lab, and create a multi-layered map with geopandas. 

## LA Times Neighborhoods

* [About mapping LA neighborhoods](http://maps.latimes.com/neighborhoods/)
* [Download the data](http://boundaries.latimes.com/sets/)

In [ ]:
import geopandas as gpd

# for basemaps
import contextily as ctx

In [ ]:
# get neighborhood boundaries from the LA Times
neighborhoods = gpd.read_file('http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/la-county-neighborhoods-v5.geojson')

In [ ]:
# trim the data to the bare minimum columns
neighborhoods = neighborhoods[['name','geometry']]
neighborhoods.head()

In [ ]:
# get the layers into a web mercator projection
# reproject to web mercator
neighborhoods = neighborhoods.to_crs(epsg=3857)

In [ ]:
# plot it!
ax=neighborhoods.plot(figsize=(12,12),
                      color='gray', 
                      edgecolor='white',
                      alpha=0.5)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)


## Get Arrest Data from LA Open Data Portal
Next, we acquire the data using the socrata API. Use the socrata documentation to grab the code syntax for our arrests data.
- https://dev.socrata.com/foundry/data.lacity.org/amvf-fr72

In [ ]:
import pandas as pd
import plotly.express as px
from sodapy import Socrata

In [ ]:
# connect to the data portal
client = Socrata("data.lacity.org", None)

results = client.get("amvf-fr72", 
                     limit=5000,
                     where = "arst_date between '2020-09-01T00:00:00' and '2020-09-30T00:00:00'",
                     order='arst_date desc')

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

# print it with .sample, which gives you random rows
df.head()

In [ ]:
df.shape

In [ ]:
# columns
list(df)

## Convert data to a geodataframe

Geopandas allows us to convert different types of data into a spatial format.
- https://geopandas.org/gallery/create_geopandas_from_pandas.html

In [ ]:
# convert pandas dataframe to geodataframe
arrests = gpd.GeoDataFrame(df, 
                         crs='EPSG:4326',
                         geometry=gpd.points_from_xy(df.lon, df.lat))

In [ ]:
# get the layers into a web mercator projection
# reproject to web mercator
arrests = arrests.to_crs(epsg=3857)

In [ ]:
# map it!
ax = arrests.plot(figsize=(12,12),color='red')

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)


## Create a two layer map

- https://geopandas.org/mapping.html

In [ ]:
# first define which layers will be your "base"
base = neighborhoods.plot(figsize=(12,10),
                      color='gray', 
                      edgecolor='white',
                      alpha=0.5)

# define the layer that will go on top, and add the base layer to the `ax` argument
ax = arrests.plot(ax=base, color='red', markersize=5)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)


## Zoom to the points, not the neighborhoods

By default, the map has zoomed out to fit both layers. Sometimes, we may want to zoom to the extent of a smaller layer, in this case, the crime data.

In [ ]:
# get the bounding box coordinates for the crime data
arrests.geometry.total_bounds

In [ ]:
# shortcut to put them into their own variables
minx, miny, maxx, maxy = arrests.geometry.total_bounds
print(minx)
print(maxx)
print(miny)
print(maxy)


In [ ]:
# use the bounding box coordinates to set the x and y limits
base = neighborhoods.plot(figsize=(12,12),
                          color='gray', 
                          edgecolor='white',
                          alpha=0.5)

ax = arrests.plot(ax=base, 
                color='red', 
                markersize=5
               )

ax.set_xlim(minx - 1000, maxx + 1000) # added/substracted value is to give some margin around total bounds
ax.set_ylim(miny - 1000, maxy + 1000)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

ax

# Function time

Our research would like to look at crime at the neighborhood level, rather than as a city as a whole. It would be very useful to have a function that zooms in to a specified neighborhood, and displayed the crime in that neighborhood. Let's create such a function.

Before we create the function, let's experiment with the code that is necessary to create the output for a **single** neighborhood. Let's use "Downtown" for our experiment.

Here are the components of the desired map:

- zoom to a specified neighborhood
- show an outline of the neighborhood
- show arrest data
- add a legend for arrest data type

In [ ]:
# subset the neighborhoods geodataframe for a single neighborhood
neighborhood = neighborhoods[neighborhoods.name=='Downtown']

# use the bounding box coordinates to set the x and y limits
minx, miny, maxx, maxy = neighborhood.geometry.total_bounds

# do a spatial join to get crime in neighborhood
arrests_in_neighborhood = gpd.sjoin(arrests,neighborhood,how='inner')

# define the base layer to be the neighborhood polygon
base = neighborhood.plot(figsize=(12,12),
                         color='red', 
                         edgecolor='red',
                         alpha=0.1)

# add the crime data, making sure to add the neighborhood polygon
ax = arrests_in_neighborhood.plot(ax=base, 
                                column='grp_description', 
                                markersize=40, 
                                legend=True,
                                cmap='tab20',
                                legend_kwds={
                                   'loc': 'upper right',
                                   'bbox_to_anchor':(1.3,1)
                                }                  # this puts the legend to the side
                            )

# set the map extent to the extent of the neighborhood bounds
ax.set_xlim(minx - 200, maxx + 200) # added/substracted value is to give some margin around total bounds
ax.set_ylim(miny - 200, maxy + 200)

# turn off the axis
ax.axis('off')

# add a title
ax.set_title('September 2020 Arrests in '+neighborhood.name.values[0]+' Los Angeles',fontsize=20)

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)
ax

Now that you know the code above generates the desired output, convert it into a function. Try it in the cell below, by adding the code above below the function declaration `def map_crime_by_neighborhood(place='Downtown'):` and make sure to indent the content inside the function (you can select multiple lines and hit the tab key).

Finally, make sure to replace `'Downtown'` with `place` (without the single quotes) to ensure that the variable for place is passed onto the functon.

Here is a [google slide](https://docs.google.com/presentation/d/e/2PACX-1vTkIEvjQoEvpBP3dmVPStBz1jCCNq4r6DRC9p2P4n_MdARhobk6pwppQpmijzDOZmzvVx49oeaIVXfX/pub?start=false&loop=false&delayms=3000) that describes this function's components.


In [ ]:
def map_arrests_by_neighborhood(place='Downtown'):
    
    # subset the neighborhoods geodataframe for a single neighborhood
    neighborhood = neighborhoods[neighborhoods.name==place]

    # use the bounding box coordinates to set the x and y limits
    minx, miny, maxx, maxy = neighborhood.geometry.total_bounds

    # do a spatial join to get crime in neighborhood
    arrests_in_neighborhood = gpd.sjoin(arrests,neighborhood,how='inner')

    # define the base layer to be the neighborhood polygon
    base = neighborhood.plot(figsize=(12,12),
                             color='red', 
                             edgecolor='red',
                             alpha=0.1)

    # add the crime data, making sure to add the neighborhood polygon
    ax = arrests_in_neighborhood.plot(ax=base, 
                                    column='grp_description', 
                                    markersize=40, 
                                    legend=True,
                                    cmap='tab20',
                                    legend_kwds={
                                       'loc': 'upper right',
                                       'bbox_to_anchor':(1.3,1)
                                    }                  # this puts the legend to the side
                                )

    # set the map extent to the extent of the neighborhood bounds
    ax.set_xlim(minx - 200, maxx + 200) # added/substracted value is to give some margin around total bounds
    ax.set_ylim(miny - 200, maxy + 200)

    # turn off the axis
    ax.axis('off')

    # add a title
    ax.set_title('September 2020 Arrests in '+neighborhood.name.values[0]+' Los Angeles',fontsize=20)

    # add a basemap
    ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)
    ax

In [ ]:
def map_arrests_by_neighborhood(place='Downtown'):
    
    # function code
    
   

In [ ]:
# call the function!
map_arrests_by_neighborhood(place='Hollywood')

## Adding more arguments

So far, our function accepts a single argument for `place.` What if we also wanted to provide the option on how to color code the arrest points? We can add another argument for `column`:

In [ ]:
def map_arrests_by_neighborhood(place='Downtown', column='grp_description'): # added another argument here
    # subset the neighborhoods geodataframe for a single neighborhood
    neighborhood = neighborhoods[neighborhoods.name==place]

    # use the bounding box coordinates to set the x and y limits
    minx, miny, maxx, maxy = neighborhood.geometry.total_bounds

    # do a spatial join to get crime in neighborhood
    arrests_in_neighborhood = gpd.sjoin(arrests,neighborhood,how='inner')

    # define the base layer to be the neighborhood polygon
    base = neighborhood.plot(figsize=(12,12),
                             color='red', 
                             edgecolor='red',
                             alpha=0.2)

    # add the crime data, making sure to add the neighborhood polygon
    ax = arrests_in_neighborhood.plot(ax=base, 
                                    column=column, ## notice that the column is now dependent on the variable being fed
                                    markersize=40, 
                                    legend=True,
                                    cmap='tab20',
                                    legend_kwds={
                                       'loc': 'upper right',
                                       'bbox_to_anchor':(1.3,1)
                                    }                  # this puts the legend to the side
                                )

    # set the map extent to the extent of the neighborhood bounds
    ax.set_xlim(minx - 200, maxx + 200) # added/substracted value is to give some margin around total bounds
    ax.set_ylim(miny - 200, maxy + 200)

    # turn off the axis
    ax.axis('off')

    # add a title
    ax.set_title('September 2020 Crime Incidents in '+neighborhood.name.values[0]+' Los Angeles',fontsize=20)

    # add a basemap
    ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)
    ax

In [ ]:
# run the function
map_arrests_by_neighborhood(place='Hollywood',column='sex_cd')